---
title: FastAPI notes
description: Notes on FastAPI
date: 2024-09
categories: [Programming]
---

- Build APIs based on standard Python type hints
- Automatically generate interactive documentation
- Fast to code, fewer bugs

In [3]:
#! pip install fastapi
#! pip install "uvicorn[standard]"

- If the following contents are in main.py, can run via `uvicorn main:app --reload`.  
    - `main` refers to main.py and app refers to the object inside main.py.
    - `--reload` reloads the page upon changes, to be used during dev, not prod.
- Can see documentation conforming to OpenAPI standard in `http://127.0.0.1:8000/docs`, from which you can use the endpoints!
- `http://127.0.0.1:8000/redoc` returns documentation in alternative format.
- Use `async def` to make the functions non-blocking, enabling other tasks to run concurrently. Useful when function performs I/O-bound operations, such as database queries, file I/O, or network requests, and when need to handle a large number of concurrent requests efficiently.
- Type hints will be validated with Pydantic, so if use a non-int in `/items/{item_id}`, will get an error.
- Order matters: If `read_user_current` is placed *after* `read_user`, will get an error since FastAPI will read functions top-down and try to validate input to be an integer.
- Use `Enums` if path parameter must come from a certain list of values.  If improper parameter is passed, FastAPI will list available values!
- To have paths be read correctly, use `:path` path converter, allowing the parameter to capture the entire path, including slashes.
- `read_animal` without additional parameters will read off animals 0-10.  With additional parameters, can specify which ones we want via *query parameters*, as in http://127.0.0.1:8000/animals/?skip=0&limit=2.  Here, ? denotes start of query parameters and & separates them.  Can also pass optional parameter as http://127.0.0.1:8000/animals/?skip=0&limit=2&optional_param=3, just make sure to specify it as typing.Optional.
- Can pass and use optional parameters as in `read_user_item`.
- Request body is data sent by client to the API and response body is data sent from API to client.  Use Pydantic to specify request body with POST request type.
    - To send a post request, could test it out in /docs or with curl -X POST "http://127.0.0.1:8000/books/" -H "Content-Type: application/json" -d '{
    "name": "The Great Gatsby",
    "author": "F. Scott Fitzgerald",
    "description": "A novel set in the 1920s",
    "price": 10.99
}'
    - Then can go to /books endpoint to see the books printed.


In [6]:
from fastapi import FastAPI
from enum import Enum
import typing as t
from pydantic import BaseModel

app = FastAPI()

@app.get("/") #route/endpoint
def home_page():
    return {"message":"Hello World!"}

@app.get("/items/{item_id}") #item_id is the path parameter
async def read_item(item_id: int):
    return {"item_id":item_id}

@app.get("/users/me") # will not work if placed after, must be before to be valid
async def read_user_current():
    return {"user_id":"Current user"}

@app.get("/users/{user_id}") 
async def read_user(user_id: int):
    return {"user_id":user_id}

class ModelName(str,Enum):
    ALEXNET = 'ALEXNET'
    RESNET = 'RESNET'
    LENET = 'LENET'

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name == ModelName.ALEXNET:
        return {'model_name':model_name}
    elif model_name.value == "LENET":
        return {'model_name': model_name}
    else:
        return {'model_name':f"You have selected {model_name.value}"}
    
@app.get("files/{file_path:path}")
async def read_file(file_path:str):
    return {"file_path":file_path}

animal_db = [{"animal_name":'cat'},{"animal_name":'llama'},{"animal_name":'alpaca'}]

@app.get("/animals/")
async def read_animal(skip: int=0, limit: int=10, optional_param: t.Optional[int]=None):
    return {"animals": animal_db[skip:skip+limit], "optional_parameter":optional_param}

@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(
    user_id: int, item_id: int, q: t.Optional[str]=None, short:bool=False
):
    item = {"item_id":item_id, "owner_id":user_id}
    if q:
        item.update({"q":q})
    if not short:
        item.update({'description':'great item with long description'})
    return item

books_db = []
class Book(BaseModel):
    name:str
    author:str
    description:t.Optional[str]
    price:float

@app.post("/books/")
async def create_item(book:Book):
    books_db.append(book)
    return book 
@app.get("/books/")
async def get_books():
    return books_db